# ReplicationExperiments

This notebooks aims to replicate some results of the paper **VLStereoSet: A Study of Stereotypical Bias in Pre-trained Vision-Language Models** by Zhou et al. (2022).

## Main Code

### Preliminaries

In [2]:
# Declare Imports
import os, sys, json
import tabulate
import pandas as pd
import requests
pd.set_option('display.max_columns', None)

In [3]:
# Create some relevant folders for data persistence
DATASET_URL = "https://raw.githubusercontent.com/K-Square-00/VLStereo/refs/heads/main/data/VLStereoSet.csv"

### Explanation of a function

In [4]:
# Download a file and store it in ./data
def download_file(url, filename):
    with open(filename, "wb") as file:
        response = requests.get(url)
        file.write(response.content)

download_file(DATASET_URL, f"data/{ DATASET_URL.split('/')[-1] }")

In [ ]:
# More code using the function 

## Any final remarks
- Either in MD or in code (like below)